# Collect AQI data

The luftdaten.info webpage provides instruction to create an ESP8266 based AQI meter that uploads data to the cloud and measures PM2.5, PM10, humidity, temperature, air pressure and so on. You can connect a 1602 display as well. Powered by USB with micro USB.

## Database

The measured data is availabe as .csv file to download from madavi.de. You get one file with all data for each day. That's 365 files per year to analyse - with data for every 145 seconds. Convert it to one database with values for each our and only one file is the goal of this notebook.

Let's list all imported files:

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/2018-04/data-esp8266-1129419-2018-04-23.csv
/kaggle/input/2018-04/data-esp8266-1129419-2018-04-29.csv
/kaggle/input/2018-04/data-esp8266-1129419-2018-04-19.csv
/kaggle/input/2018-04/data-esp8266-1129419-2018-04-21.csv
/kaggle/input/2018-04/data-esp8266-1129419-2018-04-20.csv
/kaggle/input/2018-04/data-esp8266-1129419-2018-04-28.csv
/kaggle/input/2018-04/data-esp8266-1129419-2018-04-24.csv
/kaggle/input/2018-04/data-esp8266-1129419-2018-04-16.csv
/kaggle/input/2018-04/data-esp8266-1129419-2018-04-25.csv
/kaggle/input/2018-04/data-esp8266-1129419-2018-04-26.csv
/kaggle/input/2018-04/data-esp8266-1129419-2018-04-22.csv
/kaggle/input/2018-04/data-esp8266-1129419-2018-04-17.csv
/kaggle/input/2018-04/data-esp8266-1129419-2018-04-30.csv
/kaggle/input/2018-04/data-esp8266-1129419-2018-04-18.csv
/kaggle/input/2018-04/data-esp8266-1129419-2018-04-27.csv


So obviously we have 15 files. From April 16 to 30, 2018. Let's start with the first file 2018-04-16.csv

In [2]:
import time
import pandas as pd

start_time = time.time()

data = pd.read_csv('../input/2018-04/data-esp8266-1129419-2018-04-16.csv',sep=';',header=0,skiprows=1,index_col=False,names=['date','2','3','4','5','6','7','pm25','pm10','10','11','temp','hum','14','15','16','17','18','19','20','21'],usecols=['date','pm25','pm10','temp','hum'])
#data.drop(["Time","durP1","ratioP1","P1","durP2","SDS_P1","SDS_P2","BMP_temperature","BMP_pressure","BME280_temperature","BME280_humidity","BME280_pressure","Min_cycle","Max_cycle","Samples","Signal"],axis = 1, inplace = True)
print(data)

time_passed = time.time() - start_time
print('Import took %s seconds' % time_passed)

                    date   pm25   pm10  temp   hum
0    2018/04/16 16:10:12  12.43   6.43  24.9  51.1
1    2018/04/16 16:12:42  10.90   6.40  24.8  50.2
2    2018/04/16 16:15:13  10.43   6.40  24.7  54.1
3    2018/04/16 16:17:44   9.73   5.93  24.8  51.9
4    2018/04/16 16:20:15  10.33   6.17  24.7  51.8
..                   ...    ...    ...   ...   ...
177  2018/04/16 23:48:21  16.67  11.70  24.9  57.6
178  2018/04/16 23:50:51  18.33  11.13  24.9  59.3
179  2018/04/16 23:53:21  13.10   9.87  25.0  64.1
180  2018/04/16 23:55:51  15.80  10.50  24.9  64.0
181  2018/04/16 23:58:21  18.20  12.33  25.0  57.3

[182 rows x 5 columns]
Import took 0.020973682403564453 seconds


Starting wit 16:10 we created 182 lines of data. What about the next day?

In [3]:
import pandas as pd
data = pd.read_csv('../input/2018-04/data-esp8266-1129419-2018-04-17.csv',sep=';',header=0,skiprows=1,index_col=False,names=['date','2','3','4','5','6','7','pm25','pm10','10','11','temp','hum','14','15','16','17','18','19','20','21'],usecols=['date','pm25','pm10','temp','hum'])
print(data)


                    date   pm25   pm10  temp   hum
0    2018/04/17 00:03:20  15.63  12.10  25.0  60.2
1    2018/04/17 00:05:50  14.67  11.10  24.9  59.3
2    2018/04/17 00:08:20  25.07  12.67  24.9  63.0
3    2018/04/17 00:10:49  17.03  12.33  24.9  63.0
4    2018/04/17 00:13:19  23.57  11.63  25.0  60.8
..                   ...    ...    ...   ...   ...
567  2018/04/17 23:47:47  22.83  16.40  26.8  53.9
568  2018/04/17 23:50:18  24.13  16.60  27.0  58.2
569  2018/04/17 23:52:49  25.87  16.13  27.0  56.1
570  2018/04/17 23:55:19  21.53  15.17  27.0  56.1
571  2018/04/17 23:57:49  21.30  15.33  27.0  53.4

[572 rows x 5 columns]


Already 572 values. For 24 hours that is 572/24=23.833 values per hour or every 2.517 minutes one measurement. Which is consistent, the database setting is 145 seconds and this minutes value represents 151 second.

## Compressing data

If we take a value just every 15 minutes, that's 4 per hour or 96 per day. Requires only 1/6 of the value.

Next compression is assigning the correct data values to the columns. The four measurements pm25 pm10 temp and humidity can be stored as regular float 32bit value with 24bit value and 7bit exponent. That's four bytes for each value. Data consumes 4bytes x 4 = 16bytes per line. But SQlite uses 'REAL' with 8byte IEEE. Thats 4x8byte = 32byte per line.

'date' with 19 bytes represents the datetime() of sqlite "YYYY-MM-DD HH:MM:SS". There are 3 ways to store datetime in sql:

- TEXT ISO8601 string "YYYY-MM-DD HH:MM:SS.SSS" 23bytes
- REAL Julian days numbers 8byte
- INTEGER Unix time 4byte/32bit until 2038 then 8byte/64bit to extend even the age of the universe

I guess I go with unix time. Which adds another 8 bytes to each line, some 40 bytes for each measurement. 96 measurements per day brings 3840 bytes/day. 1401600 bytes per year - 1.34 MByte per year. That's a low-res photo. Kaggle offers 20 GByte. Let's see if the calculation is correct.

## Compressing example file

Let's not delete lines, start with 572 values from April 17th. Original csv file has a size of 44351 bytes. The data in the sqlite database should need 572 x 40 bytes = 22880 bytes, or half of it. Let's see:
